### Importing the dependencies

In [13]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Data Collection and Pre-processing

In [14]:
movies_meta_data = pd.read_csv('movies_metadata.csv')
credit_data = pd.read_csv('credits.csv')
keywords_data = pd.read_csv('keywords.csv')
links_data = pd.read_csv('links.csv')

c:\Python39\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [15]:
movies_meta_data.head()
from ast import literal_eval
features = ['cast', 'crew']
for feature in features:
    credit_data[feature] = credit_data[feature].apply(literal_eval)
credit_data.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [16]:
def getActor1(ls):
    if isinstance(ls,list) and len(ls)>0:
        return ls[0]['name']
    return np.nan
def getActor2(ls):
    if isinstance(ls,list) and len(ls)>1:
        return ls[1]['name']
    return np.nan
def getActor3(ls):
    if isinstance(ls,list) and len(ls)>2:
        return ls[2]['name']
    return np.nan
def getDirector(ls):
    if isinstance(ls, list):
        for i in ls:
            if i['job']=='Director':
                return i['name']
    return np.nan

credit_data['actor_1'] = credit_data['cast'].apply(getActor1)
credit_data['actor_2'] = credit_data['cast'].apply(getActor2)
credit_data['actor_3'] = credit_data['cast'].apply(getActor3)
credit_data['director'] = credit_data['crew'].apply(getDirector)

credit_data.head()

,cast,crew,id,actor_1,actor_2,actor_3,director
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,Tom Hanks,Tim Allen,Don Rickles,John Lasseter
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,Robin Williams,Jonathan Hyde,Kirsten Dunst,Joe Johnston
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,Walter Matthau,Jack Lemmon,Ann-Margret,Howard Deutch
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,Whitney Houston,Angela Bassett,Loretta Devine,Forest Whitaker
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,Steve Martin,Diane Keaton,Martin Short,Charles Shyer


In [17]:
credit_data['id']=credit_data['id'].astype(int)
movies_meta_data = movies_meta_data[pd.to_numeric(movies_meta_data['id'], errors='coerce').notnull()]
movies_meta_data['id']=movies_meta_data['id'].astype(int)
finalDataSet = pd.merge(movies_meta_data, credit_data, on="id", how="left")
movies_meta_data.head()
credit_data.head()
finalDataSet.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,title,video,vote_average,vote_count,cast,crew,actor_1,actor_2,actor_3,director
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",Tom Hanks,Tim Allen,Don Rickles,John Lasseter
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",Robin Williams,Jonathan Hyde,Kirsten Dunst,Joe Johnston
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",Walter Matthau,Jack Lemmon,Ann-Margret,Howard Deutch
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",Whitney Houston,Angela Bassett,Loretta Devine,Forest Whitaker
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",Steve Martin,Diane Keaton,Martin Short,Charles Shyer


In [18]:
# finalDataSet['genres'] = finalDataSet['genres'].apply(literal_eval)
finalDataSet.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,title,video,vote_average,vote_count,cast,crew,actor_1,actor_2,actor_3,director
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",Tom Hanks,Tim Allen,Don Rickles,John Lasseter
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,Jumanji,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",Robin Williams,Jonathan Hyde,Kirsten Dunst,Joe Johnston
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,Grumpier Old Men,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",Walter Matthau,Jack Lemmon,Ann-Margret,Howard Deutch
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,Waiting to Exhale,False,6.1,34.0,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",Whitney Houston,Angela Bassett,Loretta Devine,Forest Whitaker
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,Father of the Bride Part II,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",Steve Martin,Diane Keaton,Martin Short,Charles Shyer


In [41]:
dct = {}
ln = {}

def f(x):
    try:
        return literal_eval(str(x))   
    except Exception as e:
        return np.nan

finalDataSet['genres'] = finalDataSet['genres'].apply(f)

def allGenre(ls):
    if isinstance(ls,list):
        for i in ls:
            if i['name'] in dct:
                dct[i['name']] += 1
            else:
                dct[i['name']] = 1
                
def allLanguage(lang):
    if lang != None:    
        if lang in ln:
            ln[lang] += 1
        else:
            ln[lang] = 1
                
finalDataSet['genres'].apply(allGenre)
finalDataSet['original_language'].apply(allLanguage)
df = pd.DataFrame.from_dict(dct, orient="index")
df.to_csv("genres.csv")
df = pd.DataFrame.from_dict(ln, orient="index")
df.to_csv("languages.csv")

{'Animation': 1998, 'Comedy': 13467, 'Family': 2831, 'Adventure': 3587, 'Fantasy': 2385, 'Romance': 6923, 'Drama': 20809, 'Action': 6730, 'Crime': 4387, 'Thriller': 7794, 'Horror': 4762, 'History': 1427, 'Science Fiction': 3140, 'Mystery': 2541, 'War': 1348, 'Foreign': 1681, 'Music': 1630, 'Documentary': 4050, 'Western': 1059, 'TV Movie': 793}
{'Animation': 1998, 'Comedy': 13467, 'Family': 2831, 'Adventure': 3587, 'Fantasy': 2385, 'Romance': 6923, 'Drama': 20809, 'Action': 6730, 'Crime': 4387, 'Thriller': 7794, 'Horror': 4762, 'History': 1427, 'Science Fiction': 3140, 'Mystery': 2541, 'War': 1348, 'Foreign': 1681, 'Music': 1630, 'Documentary': 4050, 'Western': 1059, 'TV Movie': 793}


In [40]:
# finalDataSet.loc[finalDataSet['original_language'] == 'hi']
dct = {}
ln = {}
def allActors(lang):
    if lang != None:    
        if lang in ln:
            ln[lang] += 1
        else:
            ln[lang] = 1
                
def allDirectors(lang):
    if lang != None:    
        if lang in ln:
            ln[lang] += 1
        else:
            ln[lang] = 1
            
finalDataSet['actor_1'].apply(allActors)
finalDataSet['actor_2'].apply(allActors)
finalDataSet['actor_3'].apply(allActors)

df = pd.DataFrame.from_dict(ln, orient="index")
df.to_csv("actors.csv")

finalDataSet['director'].apply(allDirectors)

df = pd.DataFrame.from_dict(ln, orient="index")
df.to_csv("directors.csv")
# print(dct,ln)

In [21]:
finalDataSet = pd.merge(finalDataSet, keywords_data, on="id", how="left")
finalDataSet.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,video,vote_average,vote_count,cast,crew,actor_1,actor_2,actor_3,director,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",Tom Hanks,Tim Allen,Don Rickles,John Lasseter,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,False,6.9,2413.0,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",Robin Williams,Jonathan Hyde,Kirsten Dunst,Joe Johnston,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,False,6.5,92.0,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",Walter Matthau,Jack Lemmon,Ann-Margret,Howard Deutch,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,False,6.1,34.0,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",Whitney Houston,Angela Bassett,Loretta Devine,Forest Whitaker,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,False,5.7,173.0,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",Steve Martin,Diane Keaton,Martin Short,Charles Shyer,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [22]:
dct = {}

finalDataSet['keywords'] = finalDataSet['keywords'].apply(f)

def getKeywords(ls):
    if isinstance(ls,list):
        for i in ls:
            if i['name'] in dct:
                dct[i['name']] += 1
            else:
                dct[i['name']] = 1
                
finalDataSet['keywords'].apply(getKeywords)

df = pd.DataFrame.from_dict(dct, orient="index")
df.to_csv("keywords_data.csv")
df.head()

,0
jealousy,197
toy,19
boy,68
friendship,423
friends,167


In [23]:
ln = {}
def getMovies(lang):
    if lang != None:    
        if lang in ln:
            ln[lang] += 1
        else:
            ln[lang] = 1
                
finalDataSet['original_title'].apply(getMovies)

df = pd.DataFrame.from_dict(ln, orient="index")
df.to_csv("allMovies.csv")
df.head()

,0
Toy Story,1
Jumanji,1
Grumpier Old Men,1
Waiting to Exhale,1
Father of the Bride Part II,1


In [26]:
finalDataSet['genresLs'] = finalDataSet['genres'].apply(f)
def normalizeJson(ls):
    if isinstance(ls,list):
        myStr = ''
        for i in ls:
            myStr += i['name'].lower() + ' '
        return myStr
    return ''

finalDataSet['genresLs'] = finalDataSet['genresLs'].apply(normalizeJson)
finalDataSet.head()
finalDataSet.to_csv("finalDataSet.csv")

In [35]:
finalDataSet.keys()

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew', 'actor_1', 'actor_2',
       'actor_3', 'director', 'keywords', 'genresLs'],
      dtype='object')